# Rappel du jour 2 

* tests
    * pyramide de tests
* manipulations de types / structures de données
    * utilisation de `fstrings`
    * dictionnaires et sets 
* début de programmation fonctionnelle
    * les fonctions sont des objets comme les autres 

In [8]:
def toto():
    global tata 
    tata += 1 
    print(id(tata), tata)

In [9]:
tata = 3.234432
print(id(tata), tata)
toto()
print(id(tata), tata)

4429620304 3.234432
4429617904 4.234432
4429617904 4.234432


In [15]:
fs = []
for i in range(5):
    def f(x, nombre=i):
        return x + nombre
    fs.append(f)

print(i)
for f in fs:
    print(f(10))

4
10
11
12
13
14


In [25]:
def spy_list():
    print("dans spy list")
    return []

print("entre spy et append wrong")

def append_wrong(val, li=spy_list()):
    print("dans execution de append wrong")
    li.append(val)
    return li

print("apres append wrong")


entre spy et append wrong
dans spy list
apres append wrong


In [32]:
append_wrong(21)

dans execution de append wrong


[20, 20, 20, 20, 20, 20, 21]

In [44]:
def append_ok(val, li=None):
    if li is None:
        li = []
    print("dans execution de append wrong")
    li.append(val)
    return li

In [47]:
append_ok(3, [1, 2])

dans execution de append wrong


[1, 2, 3]

In [65]:
s1 = "3+2" #=> 5
s2 = "3-2" #=> 1
s3 = "3/2"
left, operator,right = s3

def compute_addition(a, b):
    return int(a) + int(b)

def compute_substraction(a, b):
    return int(a) - int(b) 

def unknown_operator(operator):
    raise NotImplementedError(f"{operator=} not supported")

if operator == "+":
    res = compute_addition(left, right)
elif operator == "-":
    res = compute_substraction(left, right)

################

operations = {
    "+": compute_addition,
    "-": compute_substraction
}

operations.get(operator, lambda left, right: unknown_operator(operator))(left, right)

NotImplementedError: operator='/' not supported

In [67]:
def f(*args, **kwargs):
    print(f"{args=}, {kwargs=}")


f(1, 2, c=3, toto="riri")

args=(1, 2), kwargs={'c': 3, 'toto': 'riri'}


# Attributs de functions

In [68]:
def f():
    print("toto")

f.a = 2
print(f.a)

2


In [72]:
# https://stackoverflow.com/questions/67603536/cannot-set-function-attribute-inside-a-decorator

def deco(a_deco):
    def wrapper():
        a_deco()
        wrapper.test = 1
    return wrapper

@deco
def f():
    pass 

f()
print(f.test)

1


In [83]:
def pain(f):
    def wrapper(*args, **kwargs):
        print("pain")
        res = f(*args, **kwargs)
        print("pain")
        return res
    return wrapper 

def tomate(f):
    def wrapper(*args, **kwargs):
        res = f(*args, **kwargs)
        print("tomate")
        return res
    return wrapper 

def fromage(f):
    def wrapper(*args, **kwargs):
        print("fromage")
        res = f(*args, **kwargs)
        return res 
    return wrapper 

@pain
@tomate
@fromage
def sandwich(viande="steack"):
    print(viande)
    return viande

In [84]:

sandwich("végé")

pain
fromage
végé
tomate
pain


'végé'

In [91]:
import functools

def ingredient(nom_ingredient, dessus=False, dessous=False):
    def mon_ingredient(f):
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            if dessus:
                print(nom_ingredient)
            res = f(*args, **kwargs)
            if dessous:
                print(nom_ingredient)
            return res 
        return wrapper 
    return mon_ingredient

@ingredient("pain", dessus=True, dessous=True)
@ingredient("tomate", dessous=True)
@ingredient("fromage", dessus=True)
def sandwich(viande="steack"):
    """un joli sandwich"""
    print(viande)
    return viande


sandwich()
help(sandwich)

pain
fromage
steack
tomate
pain
Help on function sandwich in module __main__:

sandwich(viande='steack')
    un joli sandwich



In [92]:
def retry(f):
    def wrapper(*args, **kwargs):
        while True:
            try:
                f(*args, **kwargs)                
            except:
                pass
            else:
                break 

# Functools cache

In [120]:
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [121]:
%timeit fib(11)

7.63 µs ± 51.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [122]:
%timeit fib(12)

12.4 µs ± 59.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [133]:
from functools import lru_cache

@lru_cache(maxsize=10)
def fib(n):
    print(n)
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)


In [127]:
fib(10)

55

In [128]:
%timeit fib(11)

24.8 ns ± 0.228 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [129]:
%timeit fib(12)

24.7 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [130]:
%timeit fib(40)

25.1 ns ± 0.347 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [135]:
fib(100)

100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41


354224848179261915075

In [136]:
fib.cache_info()

CacheInfo(hits=99, misses=101, maxsize=10, currsize=10)

In [161]:
def my_cache(f):
    cache = {}
    def wrapper(*args):
        if args not in cache:
            res = f(*args)
            cache[args] = res
        return cache[args]
    wrapper.cache = cache # on passe le cache en attribut de notre fonction
    return wrapper

@my_cache
def fibo(n):
    if n < 2:
        return n
    return fibo(n-1) + fibo(n-2)

@my_cache
def toto(n):
    return 12


In [157]:
fibo(10)

55

In [158]:
toto(5)

12

In [159]:
fibo(5)

5

In [160]:
toto.cache

{(5,): 12}

In [152]:
fibo.cache

{(1,): 1,
 (0,): 0,
 (2,): 1,
 (3,): 2,
 (4,): 3,
 (5,): 5,
 (6,): 8,
 (7,): 13,
 (8,): 21,
 (9,): 34,
 (10,): 55}

# Slicing 

In [169]:
a = list(range(30))
s = slice(0, 15, 2)
a[s] == a[0:15:2]

True

In [172]:
a[::-1]

[29,
 28,
 27,
 26,
 25,
 24,
 23,
 22,
 21,
 20,
 19,
 18,
 17,
 16,
 15,
 14,
 13,
 12,
 11,
 10,
 9,
 8,
 7,
 6,
 5,
 4,
 3,
 2,
 1,
 0]

# Itérable 

In [173]:
a = [1, 2, 3]
print(a, type(a)) # [1, 2, 3] <class 'list'>

[1, 2, 3] <class 'list'>


In [174]:
it = iter(a)
print(it) # <list_iterator object at 0x7fa8359057b8>Générateurs / Itérateurs
print(type(it)) # <class 'list_iterator'>

<class 'list_iterator'>


In [175]:
next(it)

1

In [176]:
next(it)

2

In [177]:
next(it)

3

In [178]:
next(it)

StopIteration: 

In [181]:
it = iter(a)
while True:
    try:
        val = next(it)
        print(val)
    except StopIteration:
        break 

1
2
3


In [182]:
for index in range(len(a)):
    print(a[index])

1
2
3


In [185]:
a = [1, 2]
for i in a:
    pass 
    # a.append(1)
    # print(i)

## Yield

In [188]:
def example():
    print("avant 1")
    yield 1
    print("avant 2")
    yield 2
    print("avant 3")
    yield 3

In [195]:
e = example()
for element in example():
    print(element)

avant 1
1
avant 2
2
avant 3
3


In [196]:
def mymap(f, *iterables):
    print(iterables)
    for elem in zip(*iterables):
        print(elem)
        tmp = f(*elem)
        yield tmp 

In [197]:
mymap(lambda x: x*2, range(10))

<generator object mymap at 0x109384660>